In [1]:
import pandas as pd
from slugify import slugify
import json

In [2]:
# read data

raw_data = pd.read_excel('/home/abhinav/sink/D4D/data.xlsx', engine='openpyxl')
indicators = pd.read_excel('/home/abhinav/sink/D4D/Indicators.xlsx', engine='openpyxl')
dist_meta_data = {"morigaon":{"Title": "Morigaon", "About": "Morigaon district is an administrative district in the state of Assam in India. The district headquarters is located at Morigaon. The ancient place of occult Mayong is located in this district as well as Pobitora Wildlife Sanctuary.", "link": "https://morigaon.gov.in/"}, "nagaon":{"Title": "Nagaon", "About": "Nagaon is an administrative district in the Indian state of Assam. The undivided district was the most populous in the state as per 2011 census. The eastern, western, and southern fringes of the newly organized district were once ruled by different small feudal kings or their agents. It was a newly organized village system-hence called “Nagaon”, Na meaning new.", "link": "https://nagaon.assam.gov.in/"} }
dept_meta_data = {'panchayat-and-rural-development': {'Title': "Panchayat and Rural Development", 'Desc':'The Panchayat & Rural Development Department largely works to enhance livelihood opportunities, address chronic poverty, provides social security and works for economic inclusion of rural poor families. The Panchayat & Rural Development Department has been implementing a number of programmes in the rural area of the State by empowering rural poor through power of rights based law, aiming to transform the geography of poverty in the State.', 'schemes':['mgnrega']}, 
             'public-health-engineering': {'Title': "Public Health Engineering", 'Desc':'The Public Health Engineering Department has a mission to enhance quality of life of the people by ensuring sustainable safe drinking water and sanitation facilities and services along with promoting hygiene practices according to their choices and affordability.', 'schemes':['jjm']}}
scheme_meta_data = {"mgnrega": {"Name": "Mahatma Gandhi National Rural Employment Guarantee Act (MGNREGA)", "Title": "MGNREGA", "Desc": "The objective of MGNREGS is to enhance livelihood security of the rural masses. The scheme is meant to provide at least 100 days of wage employment per financial year (FY) to every rural household whose adult members are willing to do unskilled manual work. This is one of the world’s largest public sector employment programmes that provides guaranteed income through employment.", "dept_slug":"panchayat-and-rural-development", "logo": "/logo/mgnrega.webp"},
                    "jjm": {"Name": "Jal Jeevan Mission (JJM)", "Title": "JJM", "Desc": "The National Rural Drinking Water Programme / Jal Jeevan Mission is an initiative of the Government of India under the Ministry of Jal Shakti. It aims to provide safe and adequate drinking water to all households in rural India through provision of individual household tap connections by the year 2024.", "dept_slug":"public-health-engineering", "logo": "/logo/nhm.png"}}

In [3]:
# structure columns and filter explorer level indicators

def get_type_title(x):
    
    if x == "district_profile":
        return "District Profile"
    elif x == "district_performance":
        return "District Performance"
    elif x == "target":
        return "Targets"
    else:
        return x

indicators['scheme_slug'] = indicators['scheme_name'].map(lambda x: slugify(x))
indicators['indicator_type_title'] = indicators['indicator_type'].map(lambda x: get_type_title(x))

data = indicators[indicators.indicator_explorer == 'yes']

In [4]:
# create indicator list data in the required structure

base_json = {}

for index, row in data.iterrows():
    
    # insert district
    if  row.scheme_slug not in base_json:
        base_json[row.scheme_slug] = {row.indicator_type_title:[{"label": row.indicator_name, "slug": row.indicator_slug}]} 
            
    elif row.indicator_type_title not in base_json[row.scheme_slug]:
        base_json[row.scheme_slug][row.indicator_type_title] = [{"label": row.indicator_name, "slug": row.indicator_slug}]
        
    else:
        base_json[row.scheme_slug][row.indicator_type_title].append({"label": row.indicator_name, "slug": row.indicator_slug})
        
        
print (base_json)


with open('indicators_data.json', 'w') as fp:
    json.dump(base_json, fp, indent=4)


{'jjm': {'District Profile': [{'label': 'Habitations', 'slug': 'bhinh'}, {'label': 'HouseHolds', 'slug': 'bhnhh'}, {'label': 'Population', 'slug': 'bhpop'}, {'label': 'Villages', 'slug': 'bhvil'}, {'label': 'HH as on April 1, 2023', 'slug': 'hhapr'}, {'label': 'Percentage of HH with own / private connection', 'slug': 'phopc'}], 'Targets': [{'label': 'Percentage of FHTC tagged', 'slug': 'pfhtt'}, {'label': 'Percentage of HH with FHTC', 'slug': 'phftc'}, {'label': 'Percentage of schools with drinking water connection', 'slug': 'psdwc'}, {'label': 'Percentage of health centres with water connection', 'slug': 'phcwc'}], 'District Performance': [{'label': 'Percentage of 1-star villages', 'slug': 'p1svl'}, {'label': 'Percentage of 2-star villages', 'slug': 'p2svl'}, {'label': 'Percentage of 3-star villages', 'slug': 'p3svl'}, {'label': 'Percentage of 4-star villages', 'slug': 'p4svl'}, {'label': 'Percentage of 5-star villages', 'slug': 'p5svl'}, {'label': 'Percentage of approved government o

In [9]:
data.head()

,scheme_name,indicator_report,indicator_name,indicator_slug,indicator_type,indicator_name_short,indicator_selected,indicator_overview,indicator_explorer,indicator_dept,indicator_district,indicator_unit,scheme_slug,indicator_type_title
0,JJM,B1- Basic Habitation Information,Habitations,bhinh,district_profile,No of habitations,yes,yes,yes,no,yes,number,jjm,District Profile
1,JJM,B1- Basic Habitation Information,HouseHolds,bhnhh,district_profile,No of households,yes,yes,yes,no,yes,number,jjm,District Profile
3,JJM,B1- Basic Habitation Information,Population,bhpop,district_profile,Population,yes,yes,yes,no,yes,number,jjm,District Profile
4,JJM,B1- Basic Habitation Information,Villages,bhvil,district_profile,No of villages,yes,yes,yes,no,no,number,jjm,District Profile
47,JJM,J6 - Status of verification of beneficiary pro...,"HH as on April 1, 2023",hhapr,district_profile,Households as on 01/04/23,yes,yes,yes,no,no,number,jjm,District Profile
